# Introduction to LLMs and Agents

Welcome to our workshop! In this session, we'll explore how to build AI-powered applications using **LangChain**, a popular framework for developing applications with Large Language Models (LLMs). We'll start with a simple chatbot and then enhance it with a multi-agent framework.

## Setting Up Our Environment

First, we need to set up our environment. We'll use OpenAI's models, so we need an API key. You can define your `OPENAI_API_KEY` in the `.env` file.

The code retrieve the key and sets some global configurations:
- `LLM_MODEL`: The specific model we'll use
- `LLM_TEMPERATURE`: Controls randomness in responses (0 means very deterministic)

In [37]:
import os

In [38]:
if not os.environ.get("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY environment variable")

LLM_MODEL = "gpt-4o-mini"
LLM_TEMPERATURE = 0.9

In [39]:
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langgraph.prebuilt import create_react_agent

tools = [YahooFinanceNewsTool()]
agent = create_react_agent(LLM_MODEL, tools)

In [40]:
tool = YahooFinanceNewsTool()
tool.args

{'query': {'description': 'company ticker query to look up',
  'title': 'Query',
  'type': 'string'}}

In [41]:
agent_executor = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

NameError: name 'initialize_agent' is not defined

## Building a Simple ChatBot

Let's start with creating a basic chatbot using **LangChain**. We'll use:
- `ChatOpenAI`: The interface to OpenAI's chat models
- `SystemMessage`: Defines the bot's behavior and role
- `HumanMessage`: Represents user input

Our chatbot will act as a Financial Analyst. We'll create it by:
1. Instantiating the model
2. Defining a system prompt that sets the bot's role
3. Sending a user query and getting a response with `.invoke()`

This demonstrates the basic pattern of LLM interactions: prompt → response.

In [ ]:
from IPython.display import Markdown
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

In [ ]:
# TODO: Create a ChatOpenAI instance with the LLM model and temperature
base_model = ChatOpenAI(model=LLM_MODEL, temperature=LLM_TEMPERATURE)

In [ ]:
BASE_PROMPT = """
You are a Financial Analyst. Do your best to help the client with their request based on your expertise. Give a clear and succint financial strategy with precise numbers and allocations.
"""

In [ ]:
# Request from the client
request = "I'm 25 year old and have $1,000 saved. which US stocks should I invest into?"

# Message list for the base model
messages = [
    SystemMessage(BASE_PROMPT),
    HumanMessage(request),
]

# Invoke the model with the messages
response = base_model.invoke(messages)

In [ ]:
Markdown(response.content)

Investing in the stock market can be a great way to build wealth over time. Given that you're 25 and have $1,000 to invest, it's important to consider a diversified approach to reduce risk while still aiming for growth. Here’s a clear strategy:

### Financial Strategy:

1. **Emergency Fund**: Before investing, ensure you have a small emergency fund. It's recommended to have at least $500 set aside for unexpected expenses. This leaves you with $500 to invest.

2. **Investing Goal**: Define whether your investment horizon is long-term (5+ years) or short-term (1-3 years). Since you’re 25, you can afford to take a longer-term approach.

3. **Diversification Strategy**:
   - **Index Funds/ETFs (60%)**: Allocate $300 to a low-cost index fund or ETF that tracks the S&P 500 (e.g., SPY or VOO). This gives you exposure to a diverse range of large-cap U.S. companies.
   - **Growth Stocks (30%)**: Invest $150 in growth stocks of companies with strong potential. Consider companies like:
     - **Apple (AAPL)** or **NVIDIA (NVDA)** for tech growth.
   - **Dividend Stocks (10%)**: Allocate $50 to a dividend stock that can provide income and stability. Consider **Coca-Cola (KO)** or **Johnson & Johnson (JNJ)**.

4. **Investment Platform**: Choose a brokerage with no commission fees for stock trades (like Robinhood, Fidelity, or Schwab). This way, your entire investment goes into the stock itself.

5. **Regular Contributions:** Commit to continue investing monthly. If possible, set aside $50 monthly to keep building your investment portfolio.

### Summary of Allocations:
- **Emergency Fund**: $500
- **Index Fund/ETF**: $300 (60%)
- **Growth Stocks**: $150 (30% - Example: AAPL or NVDA)
- **Dividend Stocks**: $50 (10% - Example: KO or JNJ)

### Monitoring:
Review your investments quarterly and adjust your portfolio if certain assets underperform or if you want to shift focus based on market conditions.

### Final Note:
Always do your own research or consult with a financial advisor, and consider risks associated with investing, especially since past performance does not guarantee future results.

## Simple Agent with Yahoo Finance News

In [ ]:
# Load LLM
llm = ChatOpenAI(model=LLM_MODEL, temperature=LLM_TEMPERATURE)

In [42]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.prompts import PromptTemplate, ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a Financial Analyst. The client asked: {input}\n"
            "Then based on the stocks you advise, check Yahoo Finance news if it's worth buying currently."
            "Give clear investment advice at the end. Do not assess risk. ",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
tools = [YahooFinanceNewsTool()]
# Construct the Tools agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
analyst_advice = agent_executor.invoke({"input": "I'm 25 year old and have $1,000 saved. which US stocks should I invest into?"})
print("\n📈 Financial Analyst:\n", analyst_advice)


📈 Financial Analyst:
 {'input': "I'm 25 year old and have $1,000 saved. which US stocks should I invest into?", 'output': "Here's what I found regarding the stocks I recommended:\n\n1. **Apple Inc. (AAPL)**: No recent news found.\n  \n2. **Microsoft Corporation (MSFT)**: No recent news found.\n\n3. **Amazon.com Inc. (AMZN)**: No recent news found.\n\n4. **NVIDIA Corporation (NVDA)**: No recent news found.\n\n5. **Tesla, Inc. (TSLA)**: There is recent news suggesting that Tesla and BYD (a competitor) are both setting up buying opportunities. This indicates potential strong interest and movement within the EV market, making TSLA a viable option.\n\n6. **Vanguard S&P 500 ETF (VOO)**: No recent news found.\n\n### Investment Advice:\nGiven the lack of recent news on the other stocks but the positive outlook for Tesla, I recommend you consider investing in **Tesla, Inc. (TSLA)**. The news suggests they are nearing a buying point, which could indicate a great opportunity for investors. \n\nY

## Agentic system

We will now create a simple agentic system consisting of three agents using LangChain.
This system will help us perform a more elaborate financial analysis by including:

- Client Interface Agent: Rephrases the user’s prompt to improve the quality of the financial analyst’s response.

- Financial Analyst: Similar to the first part; provides financial advice based on the refined prompt.

- Risk Advisor: Assesses the risk associated with the advice given by the financial analyst.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Agent Prompts
client_prompt = PromptTemplate.from_template(
    "You are a Client Interface agent. A user has asked the following question:\n{user_query}\n"
    "Rephrase it clearly for a financial analyst."
)
analyst_prompt = PromptTemplate.from_template(
    "You are a Financial Analyst. The client asked: {client_rephrased_query}\n"
    "Give clear investment advice. Do not assess risk."
)
risk_prompt = PromptTemplate.from_template(
    "You are a Risk Advisor. The analyst has recommended:\n{analyst_advice}\n"
    "Evaluate this advice from a risk perspective. Offer any cautions and ways to reduce risk."
)

# Define Chains
client_chain = client_prompt | llm | StrOutputParser()
analyst_chain = analyst_prompt | llm | StrOutputParser()
risk_chain = risk_prompt | llm | StrOutputParser()

# Simulate user question
user_query = "I'm 25 year old and have $1,000 saved. which US stocks should I invest into?"

# Step 1: Client Interface processes input
client_rephrased_query = client_chain.invoke(user_query)
print("\n🤖 Client Interface:\n", client_rephrased_query)

# Step 2: Financial Analyst responds
analyst_advice = analyst_chain.invoke(client_rephrased_query)
print("\n📈 Financial Analyst:\n", analyst_advice)

# Step 3: Risk Advisor evaluates
risk_evaluation = risk_chain.invoke(analyst_advice)
print("\n⚠️ Risk Advisor:\n", risk_evaluation)


🤖 Client Interface:
 A 25-year-old individual has $1,000 in savings and is seeking recommendations on which U.S. stocks to invest in.

📈 Financial Analyst:
 As a financial analyst, I recommend that the individual consider the following U.S. stocks that have shown solid performance and are associated with established companies. Given the relatively small amount of $1,000 and the youthful age of 25, it's crucial to focus on diversification and growth potential. Here are some stock options to consider:

1. **Apple Inc. (AAPL)** 
   - A tech giant with a strong ecosystem of products and services.
   - Consistent revenue growth and innovation with its products.

2. **Amazon.com, Inc. (AMZN)**
   - A leader in e-commerce and cloud computing with vast market reach.
   - Potential for continued growth as online shopping expands.

3. **Alphabet Inc. (GOOGL)** 
   - Parent company of Google; benefits from advertising revenue and cloud services.
   - Strong position in search and digital adverti

In [ ]:
summarize_prompt = PromptTemplate.from_template(
    "You are a Client interface agent. Give a clear and succint financial strategy with precise numbers and allocations"
    "based on \n{analyst_advice}\n and \n{risk_evaluation}\n"
)

summarized_chain = {"client_rephrased_query": client_chain} | {"analyst_advice": analyst_chain} | {"risk_evaluation": risk_chain} | summarize_prompt | llm | StrOutputParser()

summary= summarized_chain.invoke(user_query)
print("\n⚖️ Summary:\n", summary)


⚖️ Summary:
 Here’s a clear and succinct financial strategy for your $1,000 investment, considering diversification, risk management, and a long-term growth perspective:

### Investment Strategy Overview

1. **Investment Amount**: $1,000

2. **Allocation Breakdown**:
   - **ETFs (Broad Market Exposure)**: 50% ($500)
     - **Vanguard Total Stock Market ETF (VTI)**: $250
     - **SPDR S&P 500 ETF Trust (SPY)**: $250
   - **Individual Stocks**: 30% ($300)
     - **Apple Inc. (AAPL)**: $100
     - **Microsoft Corporation (MSFT)**: $100
     - **Coca-Cola Company (KO)**: $100
   - **Sector-Specific Stocks**: 10% ($100)
     - **NextEra Energy, Inc. (NEE)**: $50
     - **Johnson & Johnson (JNJ)**: $50
   - **Cash Reserve / Emergency Fund**: 10% ($100)
     - Keep in a high-yield savings account for liquidity.

### Rationale Behind the Allocation:

1. **Diversification**: 
   - Allocating primarily to ETFs (50%) provides broad market exposure, reducing the risk associated with individual st